In [3]:
pip install segyio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 KB 3.0 MB/s eta 0:00:00


In [4]:
pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


LOAD DATA

In [7]:
import numpy as np
import os
from keras.models import model_from_json
import segyio
import matplotlib.pyplot as plt

In [ ]:
file_seismiknya = "file seismiknya.segy"
data = segyio.tools.cube(file_seismiknya)

In [ ]:
with segyio.open(file_seismiknya, ignore_geometry=True) as f:
  twt = f.samples

In [ ]:
clip = data[:, :400, 300:]

In [ ]:
time = twt
time_clip = twt[300:]

In [ ]:
print(data.shape)

LOAD MODEL

In [ ]:
json_file = open("masukin model json nya kesini.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("masukin model pretrained nya.hdf5")

APPLY MODEL

In [ ]:
OutName = "nama file output.dat"

In [ ]:
# Training Image Dimensions
n1, n2, n3 = 128, 128, 128
params = {"batch_size":1,
          "dim":(n1,n2,n3),
          "n_channels": 1,
          "shuffle": True}

In [ ]:
# set Gaussian Weights in the overlap boundaries
def getmask(os):
  sc = np.zeros((n1,n2,n3), dtype=np.single)
  sc = sc+1
  sp = np.zeros((os), dtype=np.single)
  sig = os/4
  sig = 0.5/(sig*sig)
  for ks in range(os):
    ds = ks-os+1
    sp[ks] = np.exp(-ds*ds*sig)
  for k1 in range(os):
    for k2 in range(n2):
      for k3 in range(n3):
        sc[k1] [k2] [k3] = sp[k1]
        sc[n1-k1-1] [k2] [k3] = sp[k1]
  for k1 in range(n1):
    for k2 in range(os):
      for k3 in range(n3):
        sc[k1] [k2] [k3] = sp[k2]
        sc[k1] [n2-k2-1] [k3] = sp[k2]
  for k1 in range(n1):
    for k2 in range(n2):
      for k3 in range(os):
        sc[k1] [k2] [k3] = sp[k3]
        sc[k1] [k2] [n3-k3-1] = sp[k3]
  return sc

In [ ]:
from pickle import BININT1
gx,m1,m2,m3 = data, data.shape[0], data.shape[1], data.shape[2]
os = 12  # Overlap Width
c1 = np.round((m1+os) / (n1-os) + 0.5)
c2 = np.round((m2+os) / (n2-os) + 0.5)
c3 = np.round((m3+os) / (n3-os) + 0.5)

c1 = int(c1)
c2 = int(c2)
c3 = int(c3)

p1 = (n1-os) * c1+os
p2 = (n2-os) * c2+os
p3 = (n3-os) * c3+os

gx = np.reshape(gx,(m1,m2,m3))
gp = np.zeros((p1,p2,p3), dtype=np.single)
gy = np.zeros((p1,p2,p3), dtype=np.single)
mk = np.zeros((p1,p2,p3), dtype=np.single)
gs = np.zeros((1,n1,n2,n3,1), dtype=np.single)

gp[0:m1, 0:m2, 0:m3] = gx
sc = getmask(os)

for k1 in range(c1):
  for k2 in range(c2):
    for k3 in range(c3):
      b1 = k1*n1-k1*os
      e1 = b1+n1
      b2 = k2*n2-k2*os
      e2 = b2+n2
      b3 = k3*n3-k3*os
      e3 = b3+n3
      gs[0,:,:,:,0] = gp[b1:e1, b2:e2, b3:e3]
      gs = gs-np.min(gs)
      gs = gs/np.max(gs)
      gs = gs*255

      Y = loaded_model.predict(gs, verbose=1)
      Y = np.array(Y)
      gy[b1:e1, b2:e2, b3:e3] = gy[b1:e1, b2:e2, b3:e3] + Y[0,:,:,:,0]*sc
      mk[b1:e1, b2:e2, b3:e3] = mk[b1:e1, b2:e2, b3:e3]+sc

gy = gy/mk
gy = gy[0:m1, 0:m2, 0:m3]
gy.tofile(OutName, format="%4")

VISUALIZATION + EVALUATION

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
ncolors = 256
color_array = plt.get_cmap("gray")(range(ncolors))
color_array[:,-1] = np.linspace(0.0, 0, ncolors)
color_array[250:250] = [1, 0.0, 0.0, 1]
map_object = LinearSegmentedColormap.from_list(name="faults1", colors=color_array)
plt.register_cmap(smap=map_object)

In [ ]:
gy,m1,m2,m3 = np.fromfile("nama file output.dat", dtype=np.single), data.shape[0], data.shape[1], data.shape[2]

gy = np.reshape(gy,(m1,m2,m3))

In [ ]:
print(gy.shape)

In [ ]:
gy_clip = gy[:, :400, 300:]

In [ ]:
print(gy_clip.shape)

In [ ]:
gy

PLOT DATA FULL

In [ ]:
extent = [1, 400, twt[-1], twt[0]]
z = 50
fig2 = plt.figure(figsize=(15.0, 10.0))
vmy = np.percentile(data, 99)
img1 = plt.imshow(data[:,:,z].T, cmap="Greys", vmin = -vmy, vmax=vmy, extent=extent, aspect="auto")
img2 = plt.imshow(gy[:,:,z].T, extent=extent, aspect="auto", cmap="faults1")
plt.ylim([min(time), max(time)])
plt.gca().invert_yaxis()
plt.xlabel("Trace Location")
plt.ylabel("time [ms]")
plt.show

INLINE

In [ ]:
extent = [1, 400, twt[-1], twt[300]]
inline = 100
fig2 = plt.figure(figsize=(15.0, 10.0))
vmy = np.percentile(data, 99)
img1 = plt.imshow(clip[inline,:,:].T, cmap="Greys", vmin = -vmy, vmax=vmy, extent=extent, aspect="auto")
img2 = plt.imshow(gy_clip[inline,:,:].T, extent=extent, aspect="auto", cmap="faults1")
plt.ylim([min(time), max(time)])
plt.gca().invert_yaxis()
plt.xlabel("Trace Location")
plt.ylabel("time [ms]")
plt.show